# NLP in Spacy

## The Yelp Dataset
[**The Yelp Dataset**](https://www.yelp.com/dataset_challenge/) is a dataset published by the business review service [Yelp](http://yelp.com) for academic research and educational purposes. I really like the Yelp dataset as a subject for machine learning and natural language processing demos, because it's big (but not so big that you need your own data center to process it), well-connected, and anyone can relate to it &mdash; it's largely about food, after all!

**Note:** If you'd like to execute this notebook interactively on your local machine, you'll need to download your own copy of the Yelp dataset. If you're reviewing a static copy of the notebook online, you can skip this step. Here's how to get the dataset:
1. Please visit the Yelp dataset webpage [here](https://www.yelp.com/dataset_challenge/)
1. Click "Get the Data"
1. Please review, agree to, and respect Yelp's terms of use!
1. The dataset downloads as a compressed .tgz file; uncompress it
1. Place the uncompressed dataset files (*yelp_academic_dataset_business.json*, etc.) in a directory named *data*

That's it! You're ready to go.

The data is provided in a handful of files in _.json_ format. We'll be using the following files for our demo:
- __yelp\_academic\_dataset\_business.json__ &mdash; _the records for individual businesses_
- __yelp\_academic\_dataset\_review.json__ &mdash; _the records for reviews users wrote about businesses_

The files are text files (UTF-8) with one _json object_ per line, each one corresponding to an individual data record. Let's take a look at a few examples.

In [1]:
import os
import codecs

data_directory = os.path.join('data')

businesses_filepath = os.path.join(data_directory,
                                   'yelp_academic_dataset_business.json')

with codecs.open(businesses_filepath, encoding='utf_8') as f:
    first_business_record = f.readline() 

print(first_business_record)

{"business_id": "O_X3PGhk3Y5JWVi866qlJg", "full_address": "1501 W Bell Rd\nPhoenix, AZ 85023", "hours": {"Monday": {"close": "18:00", "open": "11:00"}, "Tuesday": {"close": "18:00", "open": "11:00"}, "Friday": {"close": "18:00", "open": "11:00"}, "Wednesday": {"close": "18:00", "open": "11:00"}, "Thursday": {"close": "18:00", "open": "11:00"}, "Sunday": {"close": "18:00", "open": "11:00"}, "Saturday": {"close": "18:00", "open": "11:00"}}, "open": true, "categories": ["Active Life", "Arts & Entertainment", "Stadiums & Arenas", "Horse Racing"], "city": "Phoenix", "review_count": 29, "name": "Turf Paradise Race Course", "neighborhoods": [], "longitude": -112.0923293, "state": "AZ", "stars": 4.0, "latitude": 33.638572699999997, "attributes": {"Take-out": false, "Wi-Fi": "free", "Good For": {"dessert": false, "latenight": false, "lunch": false, "dinner": false, "brunch": false, "breakfast": false}, "Noise Level": "average", "Takes Reservations": true, "Has TV": true, "Delivery": false, "Amb

The business records consist of _key, value_ pairs containing information about the particular business. A few attributes we'll be interested in for this demo include:
- __business\_id__ &mdash; _unique identifier for businesses_
- __categories__ &mdash; _an array containing relevant category values of businesses_

The _categories_ attribute is of special interest. This demo will focus on restaurants, which are indicated by the presence of the _Restaurant_ tag in the _categories_ array. In addition, the _categories_ array may contain more detailed information about restaurants, such as the type of food they serve.

The review records are stored in a similar manner &mdash; _key, value_ pairs containing information about the reviews.

In [2]:
review_json_filepath = os.path.join(data_directory,
                                    'yelp_academic_dataset_review.json')

with codecs.open(review_json_filepath, encoding='utf_8') as f:
    first_review_record = f.readline()
    
print(first_review_record)

{"votes": {"funny": 0, "useful": 5, "cool": 2}, "user_id": "rLtl8ZkDX5vH5nAx9C3q5Q", "review_id": "fWKvX83p0-ka4JS3dc6E5A", "stars": 5, "date": "2011-01-26", "text": "My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I've ever had.  I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the b

A few attributes of note on the review records:
- __business\_id__ &mdash; _indicates which business the review is about_
- __text__ &mdash; _the natural language text the user wrote_

The _text_ attribute will be our focus today!

_json_ is a handy file format for data interchange, but it's typically not the most usable for any sort of modeling work. Let's do a bit more data preparation to get our data in a more usable format. Our next code block will do the following:
1. Read in each business record and convert it to a Python `dict`
2. Filter out business records that aren't about restaurants (i.e., not in the "Restaurant" category)
3. Create a `frozenset` of the business IDs for restaurants, which we'll use in the next step

In [16]:
import json

restaurant_ids = set()

# open the businesses file
with codecs.open(businesses_filepath, encoding='utf_8') as f:
    
    
    # iterate through each line (json record) in the file
    for business_json in f:
        
        # convert the json record to a Python dict
        business = json.loads(business_json)
        
        # if this business is not a restaurant, skip to the next one
        if u'Restaurants' not in business[u'categories']:
            continue
            
        # add the restaurant business id to our restaurant_ids set
        restaurant_ids.add(business[u'business_id'])

# turn restaurant_ids into a frozenset, as we don't need to change it anymore
restaurant_ids = frozenset(restaurant_ids)

# print the number of unique restaurant ids in the dataset
print('{:,}'.format(len(restaurant_ids)), u'restaurants in the dataset.')

5,556 restaurants in the dataset.


Next, we will create a new file that contains only the text from reviews about restaurants, with one review per line in the file.

In [17]:
intermediate_directory = os.path.join('intermediate')

review_txt_filepath = os.path.join(intermediate_directory,
                                   'review_text_all.txt')

In [5]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:
    
    review_count = 0

    # create & open a new file in write mode
    with codecs.open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:

        # open the existing review json file
        with codecs.open(review_json_filepath, encoding='utf_8') as review_json_file:

            # loop through all reviews in the existing file and convert to dict
            for review_json in review_json_file:
                review = json.loads(review_json)

                # if this review is not about a restaurant, skip to the next one
                if review[u'business_id'] not in restaurant_ids:
                    continue

                # write the restaurant review as a line in the new file
                # escape newline characters in the original review text
                review_txt_file.write(review[u'text'].replace('\n', '\\n') + '\n')
                review_count += 1

    print (u'''Text from {:,} restaurant reviews
              written to the new txt file.'''.format(review_count))
    
else:
    
    with codecs.open(review_txt_filepath, encoding='utf_8') as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print(u'Text from {:,} restaurant reviews in the txt file.'.format(review_count + 1))

Text from 153,886 restaurant reviews
              written to the new txt file.
CPU times: user 11 s, sys: 637 ms, total: 11.6 s
Wall time: 14.9 s


## spaCy &mdash; Industrial-Strength NLP in Python

![spaCy](https://s3.amazonaws.com/skipgram-images/spaCy.png)

[**spaCy**](https://spacy.io) is an industrial-strength natural language processing (_NLP_) library for Python. spaCy's goal is to take recent advancements in natural language processing out of research papers and put them in the hands of users to build production software.

spaCy handles many tasks commonly associated with building an end-to-end natural language processing pipeline:
- Tokenization
- Text normalization, such as lowercasing, stemming/lemmatization
- Part-of-speech tagging
- Syntactic dependency parsing
- Sentence boundary detection
- Named entity recognition and annotation

In the "batteries included" Python tradition, spaCy contains built-in data and models which you can use out-of-the-box for processing general-purpose English language text:
- Large English vocabulary, including stopword lists
- Token "probabilities"
- Word vectors

spaCy is written in optimized Cython, which means it's _fast_. According to a few independent sources, it's the fastest syntactic parser available in any language. Key pieces of the spaCy parsing pipeline are written in pure C, enabling efficient multithreading (i.e., spaCy can release the _GIL_).

In [18]:
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')

Let's grab a sample review to play with.

In [19]:
with codecs.open(review_txt_filepath, encoding='utf_8') as f:
    sample_review = list(it.islice(f, 8, 9))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

They have a limited time thing going on right now with BBQ chicken pizza (not sure how long it's going to last) but let me just say it was amazing.  Probably THE best BBQ Chicken pizza I have ever had.  I have tried other things too, like the tomato basil soup, and many of their sandwiches ... very good, very fresh - every time.  

The 5 stars is for the pizza, but if I were to rate Jason's Deli over all they would get about a 4.



Hand the review text to spaCy, and be prepared to wait...

In [20]:
%%time
parsed_review = nlp(sample_review)

CPU times: user 82 ms, sys: 16 ms, total: 98 ms
Wall time: 97.7 ms


...1/20th of a second or so. Let's take a look at what we got during that time...

In [21]:
print(parsed_review)

They have a limited time thing going on right now with BBQ chicken pizza (not sure how long it's going to last) but let me just say it was amazing.  Probably THE best BBQ Chicken pizza I have ever had.  I have tried other things too, like the tomato basil soup, and many of their sandwiches ... very good, very fresh - every time.  

The 5 stars is for the pizza, but if I were to rate Jason's Deli over all they would get about a 4.



Looks the same! What happened under the hood?

What about sentence detection and segmentation?

In [22]:
for num, sentence in enumerate(parsed_review.sents):
    print('Sentence {}:'.format(num + 1))
    print(sentence)
    print('')

Sentence 1:
They have a limited time thing going on right now with BBQ chicken pizza (not sure how long it's going to last) but let me just say it was amazing.  

Sentence 2:
Probably THE best BBQ Chicken pizza I have ever had.  

Sentence 3:
I have tried other things too, like the tomato basil soup, and many of their sandwiches ... very good, very fresh - every time.  



Sentence 4:
The 5 stars is for the pizza, but if I were to rate Jason's Deli over all they would get about a 4.




What about named entity detection?

In [23]:
for num, entity in enumerate(parsed_review.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    print('')

Entity 1: BBQ - ORG

Entity 2: BBQ Chicken - ORG

Entity 3: 5 - CARDINAL

Entity 4: Jason - ORG

Entity 5: Deli - NORP

Entity 6: about a 4 - DATE

Entity 7: 
 - GPE



What about part of speech tagging?

In [13]:
token_text = [token.orth_ for token in parsed_review]
token_pos = [token.pos_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_pos)),
             columns=['token_text', 'part_of_speech'])

,token_text,part_of_speech
0,They,PRON
1,have,VERB
2,a,DET
3,limited,ADJ
4,time,NOUN
5,thing,NOUN
6,going,VERB
7,on,PART
8,right,ADV
9,now,ADV


What about text normalization, like stemming/lemmatization and shape analysis?

In [24]:
token_lemma = [token.lemma_ for token in parsed_review]
token_shape = [token.shape_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_lemma, token_shape)),
             columns=['token_text', 'token_lemma', 'token_shape'])

,token_text,token_lemma,token_shape
0,They,-PRON-,Xxxx
1,have,have,xxxx
2,a,a,x
3,limited,limited,xxxx
4,time,time,xxxx
5,thing,thing,xxxx
6,going,go,xxxx
7,on,on,xx
8,right,right,xxxx
9,now,now,xxx


What about token-level entity analysis?

In [38]:
token_entity_type = [token.ent_type_ for token in parsed_review]
token_entity_iob = [token.ent_iob_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
             columns=['token_text', 'entity_type', 'inside_outside_begin'])

,token_text,entity_type,inside_outside_begin
0,They,,O
1,have,,O
2,a,,O
3,limited,,O
4,time,,O
5,thing,,O
6,going,,O
7,on,,O
8,right,,O
9,now,,O


What about a variety of other token-level attributes, such as the relative frequency of tokens, and whether or not a token matches any of these categories?
- stopword
- punctuation
- whitespace
- represents a number
- whether or not the token is included in spaCy's default vocabulary?

In [25]:
token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in parsed_review]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: u'Yes' if x else u''))
                                               
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,They,-20.0,,,,,Yes
1,have,-20.0,Yes,,,,Yes
2,a,-20.0,Yes,,,,Yes
3,limited,-20.0,,,,,Yes
4,time,-20.0,,,,,Yes
5,thing,-20.0,,,,,Yes
6,going,-20.0,,,,,Yes
7,on,-20.0,Yes,,,,Yes
8,right,-20.0,,,,,Yes
9,now,-20.0,Yes,,,,Yes
